In [1]:
import numpy as np
from scipy.stats import norm
from tensorflow import keras
from keras.layers import Dense, Input

In [78]:
S0 = 100
K = 100
sigma = 0.4
rho = 0.75
d = 1
wi = 1.0 / d
T = 1
r = 0.03

N = 10000
n = 20

Z = np.random.multivariate_normal(
        np.zeros(d), np.eye(d), N)

Cov = np.ones((d,d)) * rho
for i in range(d):
    Cov[i,i]=1
    
L = np.linalg.cholesky(Cov)

In [79]:
def f_multiBS_call(Z):
    BT = (np.sqrt(T) * Z) @ L.T
    S = S0*np.exp((r-0.5*sigma**2)*T+sigma*BT)
    return np.maximum(0.0, np.mean(S, axis=1)-K)

y = np.mean(f_multiBS_call(Z))

In [80]:
def network1(N, n, d):
    model = keras.models.Sequential()
    model.add(Input(shape=(N, d)))
    model.add(Dense(n * d, activation='relu'))
    model.add(Dense(2*n * d, activation='relu'))
    model.add(Dense(1))
    
    return model

model = network1(N,n,d)
loss = keras.metrics.mean_squared_error
model.compile(loss=loss)

In [82]:
model.fit(np.array([list(Z)]), np.array([y]), epochs=10)

Epoch 1/10
1/1 [==============================] - 0s 111ms/step - loss: 293.8590
Epoch 2/10
1/1 [==============================] - 0s 146ms/step - loss: 293.0016
Epoch 3/10
1/1 [==============================] - 0s 115ms/step - loss: 292.1359
Epoch 4/10
1/1 [==============================] - 0s 108ms/step - loss: 291.2620
Epoch 5/10
1/1 [==============================] - 0s 92ms/step - loss: 290.3802
Epoch 6/10
1/1 [==============================] - 0s 100ms/step - loss: 289.4904
Epoch 7/10
1/1 [==============================] - 0s 88ms/step - loss: 288.5923
Epoch 8/10
1/1 [==============================] - 0s 71ms/step - loss: 287.6855
Epoch 9/10
1/1 [==============================] - 0s 187ms/step - loss: 286.7701
Epoch 10/10
1/1 [==============================] - 0s 142ms/step - loss: 285.8458


In [94]:
np.array(model.layers[0].get_weights()[0]).shape

(1, 20)